<h1 style="color:green; margin-bottom:15px;font-size:30px">Solving SQL Problems using Python</h1

# Project Description

Your company decided to be on the wave and bought a large service for reading books by subscription. Your job as an analyst is to analyze the database. It contains information about books, publishers, authors, as well as user reviews of books. 

# Tasks

- Count how many books have been published since January 1, 2000;
- For each book, count the number of reviews and the average rating;
- Find the publisher that has released the largest number of books thicker than 50 pages;
- Identify the author with the highest average book rating - consider only books with 50 or more ratings;
- Calculate the average number of reviews from users who have given more than 50 ratings.

# Content
1. Database Connection
2. Data Import and Analysis
3. Data Description
4. Problem Solving
5. Conslusion

## 1. Database Connection

In [1]:
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student', 
 'pwd': 'Sdf4$2;d-d30pp', 
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, 
 'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## 2. Data Import and Analysis

### 2.1 Authors table

In [2]:
query = ''' SELECT *  FROM authors'''
authors = pd.io.sql.read_sql(query, con=engine)

print(authors.head(5))
print(authors.info())

   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5        Alan Moore/David   Lloyd
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None


In [3]:
#Проверим на дубликаты
print('Unique authors:', authors['author'].nunique())
print('Unique author_ids:', authors['author_id'].nunique())

Unique authors: 636
Unique author_ids: 636


There are no gaps and duplicates among the authors

### 2.2 Books table

In [4]:
query = ''' SELECT *  FROM books'''
books = pd.io.sql.read_sql(query, con=engine)
books['publication_date'] = pd.to_datetime(books['publication_date'])
print(books.head(5))
print(books.info())

   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   book_id           1000 non-null   int64         


In [5]:
print('Unique book_ids:', books['book_id'].nunique())
print('Unique author_ids:', books['author_id'].nunique())
print('Unique titles:', books['title'].nunique())
print('Unique publisher_ids:', books['publisher_id'].nunique())

Unique book_ids: 1000
Unique author_ids: 636
Unique titles: 999
Unique publisher_ids: 340


In [6]:
books.duplicated(subset='title',keep=False).sort_values()

0      False
658    False
659    False
660    False
661    False
       ...  
339    False
327    False
999    False
425     True
426     True
Length: 1000, dtype: bool

In [7]:
books[books['book_id'] == 427]

,book_id,author_id,title,num_pages,publication_date,publisher_id
426,427,39,Memoirs of a Geisha,503,2005-11-22,311


In [8]:
books[books['book_id'] == 426]

,book_id,author_id,title,num_pages,publication_date,publisher_id
425,426,39,Memoirs of a Geisha,434,2005-11-15,241


In [9]:
books['publication_date'].describe()

<ipython-input-9-6895b3c221c5>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  books['publication_date'].describe()


count                    1000
unique                    618
top       2006-08-29 00:00:00
freq                       10
first     1952-12-01 00:00:00
last      2020-03-31 00:00:00
Name: publication_date, dtype: object

There are no duplicates or gaps in the books table. The table contains 1000 books from 636 authors and 340 publishers. The table contains books with a publication date between 1952-12-01 and 2020-03-31. Book 'Memoirs of a Geisha' is recorded twice, under book_id 426 and 427.

### 2.3 Publishers table

In [10]:
query = ''' SELECT *  FROM publishers'''
publishers = pd.io.sql.read_sql(query, con=engine)

print(publishers.head(5))
print(publishers.info())

   publisher_id                          publisher
0             1                                Ace
1             2                           Ace Book
2             3                          Ace Books
3             4                      Ace Hardcover
4             5  Addison Wesley Publishing Company
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None


In [11]:
print('Unique publisher_ids:', publishers['publisher_id'].nunique())
print('Unique publishers:', publishers['publisher'].nunique())

Unique publisher_ids: 340
Unique publishers: 340


There are no duplicates or gaps among the publishers.

### 2.4 Reviews table

In [12]:
query = ''' SELECT *  FROM reviews'''
reviews = pd.io.sql.read_sql(query, con=engine)

print(reviews.head(5))
print(reviews.info())

   review_id  book_id       username  \
0          1        1   brandtandrea   
1          2        1     ryanfranco   
2          3        2       lorichen   
3          4        3  johnsonamanda   
4          5        3    scotttamara   

                                                text  
0  Mention society tell send professor analysis. ...  
1  Foot glass pretty audience hit themselves. Amo...  
2  Listen treat keep worry. Miss husband tax but ...  
3  Finally month interesting blue could nature cu...  
4  Nation purpose heavy give wait song will. List...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None


In [13]:
print('Unique review_ids:', reviews['review_id'].nunique())
print('Unique book_ids:', reviews['book_id'].nunique())
print('Unique usernames:', reviews['username'].nunique())

Unique review_ids: 2793
Unique book_ids: 994
Unique usernames: 160


The table with reviews contains 2793 reviews on 994 books from 160 users, no gaps or duplicates.

### 2.5 Ratings table

In [14]:
query = ''' SELECT *  FROM ratings'''
ratings = pd.io.sql.read_sql(query, con=engine)

print(ratings.head(5))
print(ratings.info())

   rating_id  book_id       username  rating
0          1        1     ryanfranco       4
1          2        1  grantpatricia       2
2          3        1   brandtandrea       5
3          4        2       lorichen       3
4          5        2    mariokeller       2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None


In [15]:
print('Unique rating_ids:', ratings['rating_id'].nunique())
print('Unique book_ids:', ratings['book_id'].nunique())
print('Unique usernames:', ratings['username'].nunique())
print('Unique ratings:', ratings['rating'].nunique())

Unique rating_ids: 6456
Unique book_ids: 1000
Unique usernames: 160
Unique ratings: 5


The table with ratings contains 6456 ratings (from 1 to 5) from 160 users of 1000 books, no gaps or duplicates.

### 2.6 Let's check that users, books, publishers and authors are the same in different tables

In [16]:
ratings_books = ratings['book_id'].unique()
books_books = books['book_id'].unique()

pd.Series(list(set(ratings_books).intersection(set(books_books)))).count()

1000

In [17]:
ratings_username = ratings['username'].unique()
reviews_username = reviews['username'].unique()

pd.Series(list(set(ratings_username).intersection(set(reviews_username)))).count()

160

In [18]:
reviews_books = reviews['book_id'].unique()
books_books = books['book_id'].unique()

pd.Series(list(set(reviews_books).intersection(set(books_books)))).count()

994

In [19]:
ratings_books = ratings['book_id'].unique()
books_books = books['book_id'].unique()

pd.Series(list(set(ratings_books).intersection(set(books_books)))).count()

1000

In [20]:
authors_publisher = publishers['publisher_id'].unique()
publishers_publisher = books['publisher_id'].unique()

pd.Series(list(set(authors_publisher).intersection(set(publishers_publisher)))).count()

340

In [21]:
authors_author = authors['author_id'].unique()
books_author = books['author_id'].unique()

pd.Series(list(set(authors_author).intersection(set(books_author)))).count()

636

All good. Users, books, publishers and authors in different tables are the same.

## 3. Data Description

**Books table** contains information about books:
- book_id 
- author_id 
- title 
- num_pages
- publication_date
- publisher_id

**Authors table** contains information about the authors:
- author_id 
- author — author's name.

**Publishers table** contains information about publishers:
- publisher_id 
- publisher — publisher name

**Table ratings** contains information about user ratings of books:
- rating_id 
- book_id 
- username — name of the user who left the rating
- rating — book rating

**Reviews table** contains information about user reviews:
- review_id 
- book_id 
- username — review author's name;
- text — review text.


The tables do not contain gaps and duplicates, but:
- 'Memoirs of a Geisha' book is recorded twice, under book_id 426 and 427
- Reviews table contains reviews for 994 out of 1000 books.

## 4. Problem Solving

### 4.1 Count how many books have been published since January 1, 2000

Identify the author with the highest average book rating - consider only books with 50 or more ratings;
Calculate the average number of reviews from users who have given more than 50 ratings.

In [22]:
query1 = ''' SELECT COUNT(book_id) FROM books WHERE publication_date BETWEEN '2000-01-01' AND '2022-01-30' '''
task1 = pd.io.sql.read_sql(query1, con=engine)

print(task1)

   count
0    821


*821 books have been published after January 1, 2000*

### 4.2 For each book, count the number of reviews and the average rating

In [23]:
query2 = ''' 

SELECT books.title, count(distinct reviews.review_id) as reviews_count, avg(ratings.rating) as avg_rating
FROM books
LEFT JOIN reviews on books.book_id = reviews.book_id
LEFT JOIN ratings on books.book_id = ratings.book_id
group by books.title
order by reviews_count DESC

'''
task2 = pd.io.sql.read_sql(query2, con=engine)
print(task2)

                                                 title  reviews_count  \
0                                  Memoirs of a Geisha              8   
1                              Twilight (Twilight  #1)              7   
2    Harry Potter and the Chamber of Secrets (Harry...              6   
3                                     The Glass Castle              6   
4                                      Eat  Pray  Love              6   
..                                                 ...            ...   
994    Anne Rice's The Vampire Lestat: A Graphic Novel              0   
995                            The Natural Way to Draw              0   
996   The Cat in the Hat and Other Dr. Seuss Favorites              0   
997                          Essential Tales and Poems              0   
998  Disney's Beauty and the Beast (A Little Golden...              0   

     avg_rating  
0      4.138462  
1      3.662500  
2      4.287500  
3      4.206897  
4      3.395833  
..          ...

*Books with most reviews:*
- Memoirs of a Geisha - 8, rating 4.14 (this may change once the book is recorded under one id instead of two)
- Twilight. Part 1 - 7, rating 3.66

### 4.3 Find the publisher that has released the largest number of books thicker than 50 pages

In [24]:
query3 = ''' 

SELECT publishers.publisher, count(books.book_id) as books_count
FROM books
LEFT JOIN publishers on books.publisher_id = publishers.publisher_id
where books.num_pages > 50
group by publishers.publisher
order by books_count desc
limit 1
'''
task3 = pd.io.sql.read_sql(query3, con=engine)

print(task3)

       publisher  books_count
0  Penguin Books           42


*Most books thicker than 50 books were published by Penguin Books*

### 4.4 Identify the author with the highest average book rating - consider only books with 50 or more ratings

In [25]:
query4 = ''' 

SELECT authors.author, avg(ratings.rating) as avg_rating

FROM books

INNER JOIN authors on books.author_id = authors.author_id
INNER JOIN ratings on books.book_id = ratings.book_id
INNER JOIN
(
SELECT ratings.book_id, count(ratings.rating) as marks
FROM ratings
group by ratings.book_id
order by marks desc
)
as SUBQ ON books.book_id = SUBQ.book_id

where SUBQ.marks > 50
group by authors.author
order by avg_rating desc
limit 1

'''
task4 = pd.io.sql.read_sql(query4, con=engine)

print(task4)

                       author  avg_rating
0  J.K. Rowling/Mary GrandPré    4.287097


*The author with the highest average rating for books with more than 50 ratings is J.K. Rowling (4.287)*

### 4.5 Calculate the average number of reviews from users who have given more than 50 ratings

In [26]:
query5 = ''' 

select avg(SUBQ2.reviews) from
(
select count(reviews.review_id) as reviews
from reviews
INNER JOIN
(
SELECT ratings.username, count(ratings.rating) as marks
FROM ratings
group by ratings.username
order by marks desc
)
as SUBQ ON reviews.username = SUBQ.username
where SUBQ.marks >50
group by reviews.username
)
as SUBQ2

'''
task5 = pd.io.sql.read_sql(query5, con=engine)

print(task5)

         avg
0  24.333333


*The average number of reviews from users who have given more than 50 ratings is 24.33*

## 5. Conclusions

The tables do not contain gaps and duplicates, but:
'Memoirs of a Geisha' book is recorded twice, under book_id 426 and 427

In this task, I do not have permissions to edit the DB; in real life, I would send a request to the database admin.

**Problem Solving**
1. 821 books have been published after January 1, 2000
2. Books with most reviews:
- Memoirs of a Geisha - 8, rating 4.14 (this may change once the book is recorded under one id instead of two)
- Twilight. Part 1 - 7, rating 3.66

3. Most books thicker than 50 books were published by Penguin Books
4. The author with the highest average rating for books with more than 50 ratings is J.K. Rowling (4.287)
5. The average number of reviews from users who have given more than 50 ratings is 24.33
